<a href="https://colab.research.google.com/github/EddyEjembi/LLM-Data-Trainer/blob/main/Data_Curation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U transformers accelerate datasets trl bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 10.6 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
prompt = "A model that takes in English words, and responds with a well-generated, funny, and humorous madlib poem in English. Example of how the model should perform is:\
          prompt: elephant, twirl, sparkly, London\
          response:\
          In London's streets, an elephant so fine,\
          Twirled with grace, its sparkly tutu divine.\
          It danced with joy, its trunk held high,\
          A sight to see, as it twirled by."

temperature = .4
number_of_examples = 200

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import os

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

"""# Load model directly
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")"""

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

'# Load model directly\ntokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")\nmodel = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")'

In [ ]:
import random

prev_examples = []

system_message=[
        {"role": "system", "content": f"You are generating data which will be used to train a machine learning model.\n\nYou will be given a high-level description of the model we want to train, and from that, you will generate data samples, each with a prompt/response pair.\n\nYou will do so in this format:\n```\nprompt\n-----------\n$prompt_goes_here\n-----------\n\nresponse\n-----------\n$response_goes_here\n-----------\n```\n\nOnly one prompt/response pair should be generated per turn.\n\nFor each turn, make the example slightly more complex than the last, while ensuring diversity.\n\nMake sure your samples are unique and diverse, yet high-quality and complex enough to train a well-performing model.\n\nHere is the type of model we want to train:\n`{prompt}`"},
        {"role": "user", "content": f"Generate a single data for {prompt}"},

]


input_ids = tokenizer.apply_chat_template(
    system_message,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]
for i in range(number_of_examples):
  print(f"Generating example {i+1}")
  outputs = model.generate(
      input_ids,
      max_new_tokens=256,
      eos_token_id=terminators,
      do_sample=True,
      temperature=0.6,
      top_p=0.9,
  )
  response = outputs[0][input_ids.shape[-1]:]
  print(tokenizer.decode(response, skip_special_tokens=True))
  output_string = tokenizer.decode(response, skip_special_tokens=True)
  prev_examples.append(output_string)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Generating example 1


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Here's a generated data sample:

```
prompt
-----------
space, pirate, jellyfish, beach
-----------

response
-----------
On the sun-kissed beach, a pirate so bold,
Swam through the waves, with a jellyfish to hold.
Its tentacles waving, like a pirate's flag,
It danced with glee, in the ocean's sugary lag.
The beachgoers cheered, as it twirled by,
A jellyfish pirate, in the space-tanned sky.
-----------
```

This sample includes a mix of nouns (space, pirate, jellyfish, beach) and is designed to be humorous and funny, while still being a coherent and grammatically correct madlib poem.
Generating example 2


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Here is a generated data sample:

```
prompt
-----------
galaxy, harmonica, fluffy, Paris
-----------

response
-----------
In the City of Love, a galaxy so bright,
A harmonica serenaded the Eiffel's light.
A fluffy cat strolled by, with whiskers so fine,
As the City of Love harmonized in rhyme and in time.
```

Let me know if this meets your requirements!
Generating example 198


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Here's a generated data sample for the model:

```
prompt
-----------
space, guitar, fluffy, Paris
-----------

response
-----------
In the City of Love, a space cadet strummed,
With a guitar that shone like the Eiffel's hum.
A fluffy astronaut danced with glee,
In the streets of Paris, wild and carefree.
```

Let me know if this meets your expectations!
Generating example 199


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Here is a data sample for the model:

```
prompt
-----------
galaxy, harmonica, fluffy, Paris
-----------

response
-----------
In Paris' streets, a galaxy so bright,
Played a tune on its harmonica with delight.
A fluffy cat joined in, with ears so fine,
Together they jammed, in perfect rhyme and time.
```

This sample is slightly more complex than the example provided, with a new combination of words and a different scenario. The model should generate a humorous and funny madlib poem based on the given words.
Generating example 200
Here is a generated data sample for the model:

```
prompt
-----------
space, fluffy, guitar, Hawaii
-----------

response
-----------
In Hawaii's sun-kissed beaches, a space traveler so bold,
Strummed a fluffy guitar, with chords of pure gold.
It played a tune, that echoed across the sea,
As the waves danced to the beat, of its intergalactic spree.
```

Let me know if this meets your requirements or if you'd like me to generate another one!


In [ ]:
prev_examples

["Here's a generated data sample:\n\n```\nprompt\n-----------\nspace, pirate, jellyfish, beach\n-----------\n\nresponse\n-----------\nOn the sun-kissed beach, a pirate so bold,\nSwam through the waves, with a jellyfish to hold.\nIts tentacles waving, like a pirate's flag,\nIt danced with glee, in the ocean's sugary lag.\nThe beachgoers cheered, as it twirled by,\nA jellyfish pirate, in the space-tanned sky.\n-----------\n```\n\nThis sample includes a mix of nouns (space, pirate, jellyfish, beach) and is designed to be humorous and funny, while still being a coherent and grammatically correct madlib poem.",
 "Here is a generated data sample:\n\n```\nprompt\n-----------\nquaint, astronaut, harmonica, Paris\n-----------\n\nresponse\n-----------\nIn Paris' charming streets, an astronaut so bold,\nPlayed a harmonica tune, with a quaint story to be told.\nWith a croissant in hand, and a space suit so fine,\nIt jammed with the locals, in a most peculiar rhyme.\n```\n\nLet me know if you'd lik

In [ ]:
len(prev_examples)

200

In [ ]:
def extract_data(output_string, prev_examples):
  prompts = []
  responses = []

  for example in prev_examples:
    try:
      split_example = example.split('-----------')
      prompts.append(split_example[1].strip())
      formatted_string = split_example[3].strip()
      responses.append(formatted_string.split('```')[0])
    except:
      pass
  return prompts, responses


In [ ]:
prompt, response = extract_data(output_string, prev_examples)
prompt

['space, pirate, jellyfish, beach',
 'quaint, astronaut, harmonica, Paris',
 'space, astronaut, guitar, Paris',
 'space, pineapple, robot, beach',
 'space, astronaut, disco, moon',
 'quaint, astronaut, pineapple, Paris',
 'space, disco, glittery, Paris',
 'space, disco, glitter, Paris',
 'space, robot, disco, moon',
 'space, guitar, rainbow, beach',
 'galaxy, harmonica, fluffy, Paris',
 'space, pineapple, astronaut, Hawaii',
 'space, robot, disco, beach',
 'space, bubbles, guitar, Paris',
 'space, laser, disco, Paris',
 'galaxy, harmonica, fluffy, Paris',
 'space, astronaut, pineapple, Mars',
 'space, pineapple, astronaut, moon',
 'space, laser, fluffy, Paris',
 'space, laser, disco, Paris',
 'space, robot, disco, beach',
 'astronaut, harmonica, disco, Paris',
 'space, guitar, fluffy, Paris',
 'fuzzy, pineapple, spaceship, beach',
 'giraffe, disco, glittery, Paris',
 'space, laser, fluffy, Mars',
 'whimsical, guitar, fluffy, Paris',
 'space, astronaut, harmonica, Mars',
 'space, astron

In [ ]:
response

["On the sun-kissed beach, a pirate so bold,\nSwam through the waves, with a jellyfish to hold.\nIts tentacles waving, like a pirate's flag,\nIt danced with glee, in the ocean's sugary lag.\nThe beachgoers cheered, as it twirled by,\nA jellyfish pirate, in the space-tanned sky.",
 "In Paris' charming streets, an astronaut so bold,\nPlayed a harmonica tune, with a quaint story to be told.\nWith a croissant in hand, and a space suit so fine,\nIt jammed with the locals, in a most peculiar rhyme.\n",
 "In the City of Love, a space-suited astronaut strummed,\nWith a guitar that sang of galaxies and stars' hum.\nHe twirled on the Seine, his space-boots tapping slow,\nAs the Eiffel Tower twinkled bright, with a cosmic glow.\n",
 "On a sunny beach, a robot so bright,\nDanced with flair, its pineapple in sight.\nIt spun and twirled, its metal feet aglow,\nA robot's delight, on a beach to go.\n",
 "On the moon's surface, an astronaut so cool,\nPut on their dancing shoes, and started to rule.\nTh

In [ ]:
import pandas as pd

# Create a DataFrame
df = pd.DataFrame({
    'prompt': prompt,
    'response': response
})

# Remove duplicates
df = df.drop_duplicates()

print('There are ' + str(len(df)) + ' successfully-generated examples. Here are the first few:')

df.head()

There are 200 successfully-generated examples. Here are the first few:


,prompt,response
0,"space, pirate, jellyfish, beach","On the sun-kissed beach, a pirate so bold,\nSw..."
1,"quaint, astronaut, harmonica, Paris","In Paris' charming streets, an astronaut so bo..."
2,"space, astronaut, guitar, Paris","In the City of Love, a space-suited astronaut ..."
3,"space, pineapple, robot, beach","On a sunny beach, a robot so bright,\nDanced w..."
4,"space, astronaut, disco, moon","On the moon's surface, an astronaut so cool,\n..."


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   prompt    200 non-null    object
 1   response  200 non-null    object
dtypes: object(2)
memory usage: 3.2+ KB


In [ ]:
# Split the data into train and test sets, with 90% in the train set
train_df = df.sample(frac=0.9, random_state=42)
test_df = df.drop(train_df.index)

# Save the dataframes to .jsonl files
train_df.to_json('train.jsonl', orient='records', lines=True)
test_df.to_json('test.jsonl', orient='records', lines=True)